In [1]:
import svgling

# Overview of `svgling`

### Kyle Rawlins, [kgr@jhu.edu](kgr@jhu.edu)

This package is a pure python package for rendering linguistics-style (constituent) trees in SVG. It accepts trees defined in terms of lists of lists (really, anything indexable) and strings, as well as `nltk.Tree` objects (or anything with the same API).

The `nltk` interface is described below; the list interface is pretty straightforward from bracketing notation. (In what follows, "list" is used a generic term that covers any indexable structure, including python lists and tuples.) The first element of the list is the label of a parent node, and any subsequent members of the list are daughter nodes. A parent node without a daughter node is a leaf node (as are non-list strings).

For convenient use in Jupyter notebooks, the main function to try is `svgling.draw_tree`:

In [2]:
svgling.draw_tree(("S", "NP", "VP"))

As a shortcut, the outer bracketing here can be omitted:

In [3]:
svgling.draw_tree("S", "NP", ("VP", "V"))

While multi-line nodes are generally not possible, they are allowed for leaf nodes. Simply provide the leaf node as a string that includes `\n` where a linebreak is desired. This example also shows how to get the leaf nodes to be lined up with each other. (The following example is from Carnie 2013, *Syntax: a generative introduction (3rd ed.)*, p. 93.)

In [5]:
svgling.draw_tree("TP", ("NP", "D\nThe", ("AdjP", ("AdvP", "Adv\nvery"), "Adj\nsmall"), "N\nboy"), ("VP", "V\nkissed", ("NP", "D\nthe", "N\nplatypus")), leaf_nodes_align=True)

## Spacing options

By default, `svgling` will attempt to adjust *horizontal spacing* based on the lengths of leaf nodes in a subtree. This typically produces the best results for natural language syntax trees. *Vertical spacing*, on the other hand, is for the most part a constant between-node value.

Because `svgling` does (for now) only single-pass rendering, with no javascript, some of the spacing options may involve a certain amount of heuristic adjustment (because it is impossible to know exact text sizes before actually rendering them to a device). Please let me know if you find spacing bugs and I'll try to address them.

Here are two somewhat more interesting trees, defined as lists, and various rendering options demonstrated on those trees.

In [6]:
t0 = ("S", ("NP", "D", "N"), ("VP", "V", ("NP", "D", "N")))
t1 = ("S", ("NP", ("D", "the"), ("N", "rhinoceros")), ("VP", ("V", "saw"), ("NP", ("D", "the"), ("N", "elephant"))))

Here are the default settings, which use text-width-based spacing (`HorizOptions.TEXT`):

In [7]:
svgling.draw_tree(t0)

In [8]:
svgling.draw_tree(t1)

To customize the tree layout, you provide a `svgling.core.TreeOptions` object with various parameters set. Here are the two above trees rendered using even spacing. Without manual adjustment, this may cut off longer words, but it can look better for abstract trees with similar-width leaf nodes.

In [32]:
svgling.draw_tree(t0, options=svgling.core.TreeOptions(horiz_spacing=svgling.core.HorizOptions.EVEN, leaf_padding=4))

As a convenient shortcut, if you provide named parameters to a `draw_tree` call that doesn't have an `options` parameter, they will be interpreted as options. (An options argument will override any named parameters.) I will use this shortcut to illustrate the rest of the options.

In [31]:
svgling.draw_tree(t1, horiz_spacing=svgling.core.HorizOptions.EVEN, leaf_padding=6)

The third mode uses leaf node count, irrespective of the length of the contents of the leaf node, to adjust horizontal spacing.

In [30]:
svgling.draw_tree(t0, horiz_spacing=svgling.core.HorizOptions.NODES, leaf_padding=3)

In [29]:
svgling.draw_tree(t1, horiz_spacing=svgling.core.HorizOptions.NODES, leaf_padding=5)

By default, leaf node height will be determined by the depth of the leaf node. It is possible to set leaf nodes to all align with the lowest leaf node in the tree:

In [13]:
svgling.draw_tree(t0, leaf_nodes_align=True, leaf_padding=3)

In [14]:
svgling.draw_tree(t1, leaf_nodes_align=True)

In [15]:
t2 = ("S", ("NP", ("D", "the"), ("N", "rhinoceros")), ("VP", ("V", "see", "[PAST]"), ("NP", ("D", "the"), ("N", "elephant"))))
svgling.draw_tree(t2, leaf_nodes_align=True)

Vertical spacing can be customized with `distance_to_daughter`, which stores a value in `em`s. Values less than about 1.5 are not recommended and will usually result in rendering problems. Here is an example of an extremely compact tree:

In [19]:
svgling.draw_tree(t1, leaf_nodes_align=True, distance_to_daughter=0.4, leaf_padding=0.2)

## NLTK integration

The `svgling.draw_tree` function can handle nltk.Tree objects (or any object using the same API) transparently. Be warned that complex or multi-line labels won't work very well, though.

In [20]:
import nltk
try:
    del nltk.Tree._repr_png_ # if you're on a mac and want to prevent the annoying blank tk app from showing up, do this
except:
    pass
t3 = nltk.Tree.fromstring("(S (NP (D the) (N elephant)) (VP (V saw) (NP (D the) (N rhinoceros))))")

In [21]:
svgling.draw_tree(t3)

n.b. I'm not aware of any way to get `fromstring` to allow multi-line leafs, so if you want to do this in `nltk`, you'll need to construct the tree more directly. (You can supply options for trees drawn this way as usual.)

In [22]:
from nltk import Tree
svgling.draw_tree(Tree('DP', ['D\nthe', Tree('NP', ['N\ncat'])]), leaf_nodes_align=True)

For convenience' sake, `svgling` will attempt to monkeypatch `nltk.Tree._repr_svg_` when imported (or reloaded). This means that as long as `svgling` is loaded, nltk trees should by default render using the svg format, overriding the default png-based renderer that is part of `nltk` (and its associated, fairly annoying, dependency on tk). To avoid the png rendering code altogether, you can use `del` to remove the relevant function (illustrated by the first cell in this section).

In [23]:
t3

The rendering options for trees rendered this way can be customized via `svgling.core.nltk_tree_options`, though I don't recommend messing with the spacing too much. This object is created as a default `svgling.core.TreeOptions()`, in case you need to reset it.

In [24]:
svgling.core.nltk_tree_options.leaf_nodes_align = True
t3

## Other formatting adjustments

If you really want to, you can change the font. By default it will use times with some serif as a backup. To do this supply a css-formatted string; this defaults to `"font-family: times, serif; font-weight:normal; font-style: normal;"`. I recommend supplying all of these values, as otherwise when the tree is embedded in an arbitrary document, context might impact the font choice in unexpected ways. Keep in mind also that if you are sharing your document with others, this will be dependent on installed fonts for their svg viewer.

In [25]:
svgling.core.nltk_tree_options.global_font_style = "font-family: georgia, times, serif; font-weight:normal; font-style: normal; font-size: 20pt"
t3

Here's a more extreme example; I'm not sure why you'd want to do this, but you can:

In [26]:
svgling.core.nltk_tree_options.global_font_style = "font-family: impact, times, serif; font-weight:normal; font-style: italic; font-size: 10pt"
t3

Here's something someone might actually want to do:

In [27]:
svgling.core.nltk_tree_options.global_font_style = "font-family: sans-serif; font-weight:normal; font-style: normal;"
t3

## Debug mode

Sometimes, if a tree is not rendering how you expect, it may be helpful to view it in debug mode; this draws an `em` grid along with boxes for each subtree to show how it is doing positioning. This is set with the `debug` value on options objects.

In [28]:
svgling.core.nltk_tree_options = svgling.core.TreeOptions()
svgling.core.nltk_tree_options.debug = True
t3

## Further customization

Because of the nature of SVG, various aspects of trees are very easy to customize, and various things that would be nice to have are nonetheless extremely hard to do. Even things that aren't exposed in the `svgling` API may be pretty easy to do with some light under-the-hood customization of the tree rendering code. If you have specific requests, let me know.

Things that would be easy, if anyone actually wants them:
* change colors of text, have some nodes be italic/bolded
* change line stroke / width
* other graphical customization of what is currently rendered, SVG is pretty powerful.

Things that might be possible, but not easy:
* support multi-line nodes of text, beyond just leaf nodes. This should be pretty doable, but requires some of the layout code to be generalized.
* draw movement arrows (getting the `x` positions is tricky, but not impossible.)
* draw a triangle of laziness (I haven't implemented this yet just because my main use case is actual parsed data where you can't do this, but I probably will implement it). This is only tricky because it requires estimating text width accurately to look good.
* labels along edges? SVG has pretty good support for this but I don't (currently) know how to use it, so it might actually be easy.

Things that are quite hard, but desirable (most of these need multi-pass rendering to do right):
* resize trees based on actual node sizes.
* support complex nodes in a more general way.
* put MathJax in nodes, perhaps using SVG `foreignObject`. Support `foreignObject` nodes in general. (This requires being able to calculate node sizes, too.)
* allow positioning of nodes or interactive trees via javascript.
* export to raster format. (Perhaps via inkscape?)
